# Daily COVID monitoring data for Santa Barbara county

In [1]:
! jt -t grade3 -tf robotosans -cellw 1100

This table is automatically updated with new daily COVID data for the county of Santa Barbara, CA. Every day, the California Department of Public Health publishes COVID tracking metrics for each county, but does not save the previous day's data. This tool keeps track of the daily metrics that are used to monitor progress towards safe reopening in California.

In [2]:
# Preamble
import os
from IPython.display import Markdown as md
import IPython
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [3]:
# Set working directory
os.chdir("/home/alice_lepissier/cron-job")

In [4]:
# Request content from URL
html_string = requests.get('https://www.cdph.ca.gov/Programs/CID/DCDC/Pages/COVID-19/COVID19CountyDataTable.aspx').content

In [5]:
# Parse HTML
soup = BeautifulSoup(html_string, 'html.parser')

In [6]:
# Extract data for Santa Barbara
table = soup.find_all("tr", class_="ms-rteTableEvenRow-3")
SB = table[21]

In [7]:
# Avg # tests per day (per 100,000 population)
daily_tests = SB.find_all("td", class_="ms-rteTableOddCol-3")[0].text

# Case rate per 100,000
case_rate = SB.find_all("td", class_="ms-rteTableEvenCol-3")[1].text

# Testing positivity (%)
test_positivity = SB.find_all("td", class_="ms-rteTableOddCol-3")[1].text

# % Change in 3-day avg COVID+ hospitalized patients
change_hospit = SB.find_all("td", class_="ms-rteTableEvenCol-3")[2].text

# % ICU beds currently available
ICU_beds = SB.find_all("td", class_="ms-rteTableOddCol-3")[2].text

# % Ventilators currently available
ventilators = SB.find_all("td", class_="ms-rteTableEvenCol-3")[3].text

# Extract date of last update
update_date = soup.find_all("span", class_="ms-rteStyle-Accent1")[1]\
    .find("em").text

In [8]:
# Create updated date string
update_tag = "Table last updated by CDPH on " + update_date.split(" on ", 1)[1] + "."

In [9]:
# Set up Google Sheet API
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(creds)

In [10]:
# Append daily data as a new row in Google Sheet
sheet = client.open("Test cron job").sheet1
insertRow = [datetime.date.today().strftime("%Y-%m-%d"),
             daily_tests,
             case_rate,
             test_positivity,
             change_hospit,
             ICU_beds,
             ventilators]
sheet.insert_row(insertRow, 2)

{'spreadsheetId': '1pcqdMzKtea2Oq3BDmdAOQ21gZjCssieKI0692VfNLXU',
 'updatedRange': 'Sheet1!A2:G2',
 'updatedRows': 1,
 'updatedColumns': 7,
 'updatedCells': 7}

In [11]:
# Extract data from Google Sheet as pandas dataframe
sheet = client.open("Test cron job").sheet1
data = sheet.get_all_records()
result = pd.DataFrame(data).set_index("Date")

In [12]:
# Style the pandas dataframe
d = dict(selector="th",
         props=[('text-align', 'center')])

result.reset_index().style\
    .set_properties(**{'width':'20em', 'text-align':'center'})\
    .set_properties(subset=['Date'], **{'font-weight': 'bold'})\
    .set_table_styles([d])\
    .set_precision(1)\
    .hide_index()

Date,"Daily tests (per 100,000)¹","Case rate (per 100,000)²",Testing positivity (%)³,Change in hospitalizations (%)⁴,ICU beds available (%),Ventilators (available (%)
2020-08-26,219.7,133.2,✔,✔,✔,✔
2020-08-26,219.7,133.2,✔,✔,✔,✔
2020-08-26,219.7,133.2,✔,✔,✔,✔
2020-08-26,219.7,133.2,✔,✔,✔,✔
2020-08-26,219.7,133.2,✔,✔,✔,✔
2020-08-26,219.7,133.2,✔,✔,✔,✔
2020-08-26,219.7,133.2,✔,✔,✔,✔
2020-08-26,219.7,133.2,✔,✔,✔,✔
2020-08-26,219.7,133.2,✔,✔,✔,✔
2020-08-26,219.7,133.2,✔,✔,✔,✔


<sup>Notes: (1) 7 day average with a 7-day lag. (2) 14 day average. (3) 7 day average with a 7-day lag. (4) Change in 3 day average.</sup>

In [13]:
md(update_tag)

Table last updated by CDPH on August 25, 2020.

## Tracking county closures in Santa Barbara

The state of California is using a data-driven approach informed by scientific evidence to tailor its response to COVID-19. Three dimensions of concern are monitored to assess whether a county should be placed on the County Monitoring List:
 1. Elevated Disease Transmission
 2. Increasing Hospitalization
 3. Limited Hospital Capacity

Once a county appears on the County Monitoring List for three consecutive days, indoor operations in the sectors listed in Section 3 of the [July 13th State Health Officer Order](https://www.cdph.ca.gov/Programs/CID/DCDC/CDPH%20Document%20Library/COVID-19/SHO%20Order%20Dimming%20Entire%20State%207-13-2020.pdf) must close.

This table tracks where Santa Barbara stands in terms of the criteria for inclusion on the County Monitoring List. As of 20 August 2020, the criteria for inclusion on the monitoring list are:

* Case rate >100 OR (Case rate >25 AND Positivity >8%)
* \>10% increase in hospitalizations
* <20% ICU beds available OR <25% ventilators available

If Santa Barbara meets the criteria for removal from the County Monitoring List for a certain metric, then a checkmark ✔ is displayed in the table.

Read more at <https://covid19.ca.gov/roadmap-counties/>.

Data source for county metrics: https://www.cdph.ca.gov/Programs/CID/DCDC/Pages/COVID-19/COVID19CountyDataTable.aspx.

**See the code used to scrape the data: https://github.com/walice/COVID-SB/.**

In [14]:
%%javascript
IPython.notebook.save_notebook()

<IPython.core.display.Javascript object>

In [15]:
# Convert Jupyter Notebook to HTML
! jupyter nbconvert "index.ipynb" --to html --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags="['remove_cell']" --TagRemovePreprocessor.remove_input_tags="['remove_input']" --no-prompt

[NbConvertApp] Converting notebook index.ipynb to html


[NbConvertApp] Writing 362980 bytes to index.html


In [16]:
# Create daily commit message
now = datetime.datetime.now()
commit_message = "Last scraped on " + str(now)

In [ ]:
# Deploy to GitHub
! git add .
! echo $commit_message > commit_message.txt
! git commit -F commit_message.txt
! git push